# MDT 데이터 EDA
**파일**: `20260202_봉담읍_GPOT.txt`  
**형식**: 세미콜론(`;`) 구분자, 약 26,000행

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

DATA_PATH = "/Users/youngjun/Documents/AIProject/data/Tilt/MDT_서부엔지/20260202_봉담읍_GPOT.txt"
df = pd.read_csv(DATA_PATH, delimiter=';', encoding='utf-8')
print(f"Shape: {df.shape}")
df.head()

## 1. 기본 정보

In [ ]:
print("=== 컬럼 및 타입 ===")
df.info()
print(f"\n기간_월 종류: {df['기간_월'].unique()}")
print(f"RU_ID 수: {df['RU_ID'].nunique()}")
print(f"GPOT_ID 수: {df['GPOT_ID'].nunique()}")
print(f"POT_IN/OUT 분포:\n{df['POT_IN/OUT'].value_counts()}")

In [ ]:
print("=== 결측치 ===")
missing = df.isnull().sum()
missing[missing > 0]

In [ ]:
# 수치 컬럼 기초 통계
numeric_cols = ['수집건수', 'RSRP평균', 'RSRQ평균', 'SINR평균', 'Latency평균', 'DL_RI평균', 'CQI평균', '평균거리']

# 쉼표 제거 후 숫자 변환
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

df[numeric_cols].describe().round(2)

## 2. 주요 지표 분포

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

metrics = [
    ('RSRP평균', 'RSRP (dBm)'),
    ('RSRQ평균', 'RSRQ (dB)'),
    ('SINR평균', 'SINR (dB)'),
    ('Latency평균', 'Latency (ms)'),
    ('DL_RI평균', 'DL RI'),
    ('CQI평균', 'CQI'),
]

for ax, (col, label) in zip(axes.flat, metrics):
    df[col].dropna().hist(bins=50, ax=ax, edgecolor='black', alpha=0.7)
    ax.set_title(label, fontsize=13)
    ax.axvline(df[col].median(), color='red', linestyle='--', label=f'median={df[col].median():.1f}')
    ax.legend()

plt.suptitle('주요 지표 분포', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

## 3. 셀(RU_ID)별 통계

In [ ]:
cell_stats = df.groupby('RU_ID').agg(
    격자수=('GPOT_ID', 'nunique'),
    총수집건수=('수집건수', 'sum'),
    RSRP_mean=('RSRP평균', 'mean'),
    RSRQ_mean=('RSRQ평균', 'mean'),
    SINR_mean=('SINR평균', 'mean'),
    CQI_mean=('CQI평균', 'mean'),
    Latency_mean=('Latency평균', 'mean'),
).round(2)

print(f"셀 수: {len(cell_stats)}")
cell_stats.sort_values('SINR_mean').head(20)

In [ ]:
# 셀별 격자 수 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

cell_stats['격자수'].hist(bins=30, ax=axes[0], edgecolor='black', alpha=0.7)
axes[0].set_title('셀당 격자 수 분포')
axes[0].set_xlabel('격자 수')

cell_stats['SINR_mean'].hist(bins=30, ax=axes[1], edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('셀 평균 SINR 분포')
axes[1].set_xlabel('SINR (dB)')
axes[1].axvline(10.0, color='red', linestyle='--', label='threshold=10.0')
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. 문제 격자 분석 (RSRP 양호 & RSRQ 불량)

In [ ]:
RSRP_TH = -100
RSRQ_TH = -15

df['is_problem'] = (df['RSRP평균'] >= RSRP_TH) & (df['RSRQ평균'] <= RSRQ_TH)

print(f"전체 격자: {len(df)}")
print(f"문제 격자 (RSRP>={RSRP_TH} & RSRQ<={RSRQ_TH}): {df['is_problem'].sum()} ({df['is_problem'].mean()*100:.1f}%)")

# 셀별 문제 격자 비율
cell_problem = df.groupby('RU_ID').agg(
    total=('is_problem', 'count'),
    problem=('is_problem', 'sum'),
)
cell_problem['ratio'] = (cell_problem['problem'] / cell_problem['total']).round(3)
cell_problem = cell_problem.sort_values('ratio', ascending=False)

print(f"\n문제 격자 비율 30% 이상 셀: {(cell_problem['ratio'] >= 0.3).sum()}개")
cell_problem[cell_problem['ratio'] >= 0.3].head(20)

## 5. RSRP vs RSRQ 산점도

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

scatter = ax.scatter(
    df['RSRP평균'], df['RSRQ평균'],
    c=df['SINR평균'], cmap='RdYlGn', alpha=0.3, s=5
)
plt.colorbar(scatter, label='SINR (dB)')

ax.axvline(RSRP_TH, color='blue', linestyle='--', alpha=0.7, label=f'RSRP={RSRP_TH}')
ax.axhline(RSRQ_TH, color='red', linestyle='--', alpha=0.7, label=f'RSRQ={RSRQ_TH}')

# 문제 영역 표시 (RSRP 양호 & RSRQ 불량 = 우하단)
ax.fill_between([RSRP_TH, df['RSRP평균'].max()+1], RSRQ_TH, df['RSRQ평균'].min()-1,
                alpha=0.08, color='red', label='문제 영역')

ax.set_xlabel('RSRP (dBm)')
ax.set_ylabel('RSRQ (dB)')
ax.set_title('RSRP vs RSRQ (색상: SINR)')
ax.legend()
plt.tight_layout()
plt.show()

## 6. 지리적 분포 (위도/경도)

In [ ]:
# GPOT_ID -> 위도/경도 변환 (EPSG:5179 -> EPSG:4326)
df[['POT_X', 'POT_Y']] = df['GPOT_ID'].str.split('_', expand=True).astype(float) * 25
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['POT_X'], df['POT_Y']),
    crs='EPSG:5179'
).to_crs('EPSG:4326')
df['lat'] = gdf.geometry.y
df['lon'] = gdf.geometry.x

print(f"위도 범위: {df['lat'].min():.4f} ~ {df['lat'].max():.4f}")
print(f"경도 범위: {df['lon'].min():.4f} ~ {df['lon'].max():.4f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# SINR 지도
sc1 = axes[0].scatter(df['lon'], df['lat'], c=df['SINR평균'], cmap='RdYlGn',
                      s=3, alpha=0.5, vmin=-5, vmax=30)
plt.colorbar(sc1, ax=axes[0], label='SINR (dB)')
axes[0].set_title('SINR 지리적 분포')
axes[0].set_xlabel('경도')
axes[0].set_ylabel('위도')

# 문제 격자 지도
normal = df[~df['is_problem']]
problem = df[df['is_problem']]
axes[1].scatter(normal['lon'], normal['lat'], c='gray', s=2, alpha=0.3, label='정상')
axes[1].scatter(problem['lon'], problem['lat'], c='red', s=5, alpha=0.6, label='문제 격자')
axes[1].set_title(f'문제 격자 분포 (RSRP>={RSRP_TH} & RSRQ<={RSRQ_TH})')
axes[1].set_xlabel('경도')
axes[1].set_ylabel('위도')
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. 지표 간 상관관계

In [ ]:
corr_cols = ['RSRP평균', 'RSRQ평균', 'SINR평균', 'Latency평균', 'DL_RI평균', 'CQI평균', '평균거리', '수집건수']
corr = df[corr_cols].corr().round(2)

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(corr, cmap='RdBu_r', vmin=-1, vmax=1)
plt.colorbar(im)

ax.set_xticks(range(len(corr_cols)))
ax.set_yticks(range(len(corr_cols)))
ax.set_xticklabels(corr_cols, rotation=45, ha='right')
ax.set_yticklabels(corr_cols)

for i in range(len(corr_cols)):
    for j in range(len(corr_cols)):
        ax.text(j, i, f'{corr.iloc[i, j]:.2f}', ha='center', va='center', fontsize=9)

ax.set_title('지표 간 상관관계')
plt.tight_layout()
plt.show()

## 8. INDOOR vs OUTDOOR 비교

In [ ]:
pot_stats = df.groupby('POT_IN/OUT')[['RSRP평균', 'RSRQ평균', 'SINR평균', 'CQI평균', '수집건수']].agg(['mean', 'median', 'count']).round(2)
pot_stats

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for ax, col in zip(axes, ['RSRP평균', 'RSRQ평균', 'SINR평균']):
    for label, group in df.groupby('POT_IN/OUT'):
        group[col].dropna().hist(bins=40, ax=ax, alpha=0.5, label=label, edgecolor='black')
    ax.set_title(col)
    ax.legend()

plt.suptitle('INDOOR vs OUTDOOR 지표 비교', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()